In [0]:
# COMMAND 1: Imports & Config
import requests
from pyspark.sql.functions import col, from_unixtime
import pandas as pd

API_KEY = "dbc2b01aec99b8736db5b03272821b0a"
CITY    = "Bucharest"
URL     = (f"https://api.openweathermap.org/data/2.5/weather?q={CITY}&appid={API_KEY}&units=metric")

# fetch & create DataFrame
resp       = requests.get(URL)
pdf        = pd.json_normalize(resp.json())
weather_df = spark.createDataFrame(pdf)

print("API status:", resp.status_code)
print("API data:", data)
print("API cod type:", type(data.get('cod')), "value:", data.get('cod'))

# register for SQL
weather_df.createOrReplaceTempView("weather_tmp")


# write raw JSON into Volume
(
    weather_df.write
      .format("delta")
      .mode("append")
      .save("/Volumes/city_catalog/insights/raw_volume")
)

# clean & write to Clean Volume
clean_df = weather_df.select(
    col("name").alias("city"),
    col("`main.temp`" ).alias("temperature"),
    col("`main.humidity`").alias("humidity"),
    from_unixtime(col("dt")).alias("timestamp")
)
(
    clean_df.write
      .format("delta")
      .mode("append")
      .save("/Volumes/city_catalog/insights/clean_volume")
)


In [0]:
df = spark.read.format("delta").load("/Volumes/city_catalog/insights/raw_volume")
df.display()  # or df.show()